In [1]:
import os
import gc
from chime_frb_api import frb_master
import datetime as dt
import numpy as np
from IPython.display import clear_output
import time
import pandas as pd
import json
gc.collect()

0

In [2]:
R3_fitburst_event = {'event_id': 0,
                     'fitburst_snr': 0,
                     'fitburst_dm': 0,
                     'fitburst_dm_error':0.0,
                     'num_sub_bursts':0,
                     'pulse_width_ms': [0],
                     'pulse_width_ms_error': [0],
                     'scattering_time_ms': 0,
                     'scattering_time_ms_error': 0,
                     'fitburst_amplitude': [0],
                     'fitburst_amplitude_error': [0],
                     'spectral_index': [0],
                     'spectral_index_error': [0],
                     'spectral_running': [0],
                     'spectral_running_error':[0],
                     'Bandwidth':0.0,
                     'Central Frequency':0.0,
                     'type_of_data': 'intensity'
                    }

In [3]:
R3_ids_int = [15372046, 18685272, 20425523, 20425669, 20887861, 20887865, 23491552, 
              23588210, 28293423, 28293424, 39150325, 39150809, 39275710, 39278375, 
              39278417, 40163166, 40363754, 40750656, 47313912, 47583802, 51101068, 
              51101217, 51101285, 60073222, 65511496, 65511653, 71784510, 73725034, 
              73909400, 78294947, 81931588, 82019295, 95827868, 100904168, 100905566, 
              105757295, 130061681, 131640713, 135577833, 139459007, 144520399, 154779432,
             163181404, 176156139, 188760049, 188760957, 198142107]
R3_ids_int = np.array(R3_ids_int)
print(R3_ids_int)

[ 15372046  18685272  20425523  20425669  20887861  20887865  23491552
  23588210  28293423  28293424  39150325  39150809  39275710  39278375
  39278417  40163166  40363754  40750656  47313912  47583802  51101068
  51101217  51101285  60073222  65511496  65511653  71784510  73725034
  73909400  78294947  81931588  82019295  95827868 100904168 100905566
 105757295 130061681 131640713 135577833 139459007 144520399 154779432
 163181404 176156139 188760049 188760957 198142107]


In [4]:
R3_int_pub = [139459007, 144520399, 154779432, 176156139, 183057477, 188760049, 188760957, 198142107, 201096712]
R3_int_pub = np.array(R3_int_pub)

In [5]:
R3_e_fon = [157779605, 162981697, 163181404, 165238765, 175084048, 175930797]
R3_e_fon = np.array(R3_e_fon)

In [6]:
All_event_data = []
no_lock = []
failed = []
r3_events = []
r2_events = []
for e in R3_int_pub:
    R3_fitburst_event = {'event_id': e,
                         'fitburst_snr': 0,
                         'fitburst_dm': 0,
                         'fitburst_dm_error':0.0,
                         'num_sub_bursts':0,
                         'pulse_width_ms': [0],
                         'pulse_width_ms_error': [0],
                         'scattering_time_ms': 0,
                         'scattering_time_ms_error': 0,
                         'fitburst_amplitude': [0],
                         'fitburst_amplitude_error': [0],
                         'spectral_index': [0],
                         'spectral_index_error': [0],
                         'spectral_running': [0],
                         'spectral_running_error':[0],
                         'Bandwidth':0.0,
                         'Central Frequency':0.0,
                         'UTC_400': 0.0,
                         'UTC_400_err':0.0,
                         'type_of_data': 'intensity'
                        }
    frb_master_base_url = "http://frb-vsop.chime:8001"
    master = frb_master.FRBMaster(base_url = frb_master_base_url)


    event_id = e
    print(event_id)

    event = master.events.get_event(event_number=int(event_id))
    
    burst_time_str = event['measured_parameters'][0]['datetime'][0:19]
    burst_time = dt.datetime.strptime(burst_time_str, "%Y-%m-%d %H:%M:%S")
    print(burst_time)

    year = burst_time.strftime("%Y")
    month = burst_time.strftime("%m")
    day = burst_time.strftime("%d")

    file = '/data/chime/intensity/processed/{}/{}/{}/astro_{}'.format(year, month, day, event_id)
    gc.collect()
    print(file)
    try:
        print(event['locked'])
    except:
        print("NO LOCKED DATA")
        no_lock.append(e)

    try:
        round2 = False
        round3 = False
        complete = False
        if "locked" in event:
            if "intensity-fitburst" in event["locked"]:
                locked_id = event["locked"]["intensity-fitburst"]
                print("Locked id", locked_id)
                for mp in event["measured_parameters"]:
                    if mp["pipeline"]["name"] == "intensity-fitburst":
                        if (mp["measurement_id"] == locked_id) \
                            and not (mp["pipeline"]["status"] == "ERROR"):
                            if mp["pipeline"]["logs"] == "Round 2 - ":
                                round2_results = mp
                                round2 = True
                                print(mp["beam_number"])
                            if mp["pipeline"]["logs"] == "Round 3 - ":
                                round3_results = mp
                                round3 = True
                                print(mp["beam_number"])
            # pick round 3 over round 2 as in first catalog
            if round3 \
                and "snr" in round3_results.keys() and "ftest_statistic" in round3_results.keys() \
                and round3_results["snr"] >= round2_results["snr"] \
                and round3_results["ftest_statistic"] <= 0.001 \
                and (np.array(round3_results["sub_burst_scattering_timescale"]) >= 0.0001).all() \
                and (np.array(round3_results["sub_burst_scattering_timescale"]) \
                    > 1 * np.array(round3_results["sub_burst_scattering_timescale_error"])).all():
                print("##########ROUND 3 is a better fit#########")
                r3_events.append(e)
                R3_fitburst_event['fitburst_snr'] = round3_results["snr"]
                R3_fitburst_event['fitburst_dm'] = round3_results["sub_burst_dm"][0]
                R3_fitburst_event['fitburst_dm_error'] = round3_results["sub_burst_dm_error"][0]
                R3_fitburst_event['num_sub_bursts'] = len(round3_results["sub_burst_width"])
                R3_fitburst_event['pulse_width_ms'] = round3_results["sub_burst_width"]
                R3_fitburst_event['pulse_width_ms_error'] = round3_results["sub_burst_width_error"]
                R3_fitburst_event['scattering_time_ms'] = round3_results["sub_burst_scattering_timescale"][0]
                R3_fitburst_event['scattering_time_ms_error'] = round3_results["sub_burst_scattering_timescale_error"][0]
                R3_fitburst_event['fitburst_amplitude'] = round3_results["sub_burst_fluence"]
                R3_fitburst_event['fitburst_amplitude_error'] = round3_results["sub_burst_fluence_error"]
                R3_fitburst_event['spectral_index'] = round3_results["sub_burst_spectral_index"]
                R3_fitburst_event['spectral_index_error'] = round3_results["sub_burst_spectral_index_error"]
                R3_fitburst_event['spectral_running'] = round3_results["sub_burst_spectral_running"]
                R3_fitburst_event['spectral_running_error'] = round3_results["sub_burst_spectral_running_error"]
                R3_fitburst_event['UTC_400'] = round3_results["sub_burst_timestamp_UTC"]
                R3_fitburst_event['UTC_400_err'] = round3_results["sub_burst_timestamp_error"]
                complete = True
            elif round2:
                r2_events.append(e)
                R3_fitburst_event['fitburst_snr'] = round2_results["snr"]
                R3_fitburst_event['fitburst_dm'] = round2_results["sub_burst_dm"][0]
                R3_fitburst_event['fitburst_dm_error'] = round2_results["sub_burst_dm_error"][0]
                R3_fitburst_event['num_sub_bursts'] = len(round2_results["sub_burst_width"])
                R3_fitburst_event['pulse_width_ms'] = round2_results["sub_burst_width"]
                R3_fitburst_event['pulse_width_ms_error'] = round2_results["sub_burst_width_error"]
                R3_fitburst_event['scattering_time_ms'] = 2 * min(round2_results["sub_burst_width"])
                R3_fitburst_event['fitburst_amplitude'] = round2_results["sub_burst_fluence"]
                R3_fitburst_event['fitburst_amplitude_error'] = round2_results["sub_burst_fluence_error"]
                R3_fitburst_event['spectral_index'] = round2_results["sub_burst_spectral_index"]
                R3_fitburst_event['spectral_index_error'] = round2_results["sub_burst_spectral_index_error"]
                R3_fitburst_event['spectral_running'] = round2_results["sub_burst_spectral_running"]
                R3_fitburst_event['spectral_running_error'] = round2_results["sub_burst_spectral_running_error"]
                R3_fitburst_event['UTC_400'] = round2_results["sub_burst_timestamp_UTC"]
                R3_fitburst_event['UTC_400_err'] = round2_results["sub_burst_timestamp_error"]
                complete = True
    except:
        print("EVENT FAILED")
        failed.append(e)

    
    print(R3_fitburst_event, "\n")
    All_event_data.append(R3_fitburst_event)
    gc.collect()

[2023-01-23 21:09:49,480] INFO Authorization Status: None
[2023-01-23 21:09:49,481] INFO Authorization Method: Tokens


139459007


[2023-01-23 21:09:49,767] INFO Authorization Token : Expired
[2023-01-23 21:09:49,770] INFO Reauthorize Method: Tokens
[2023-01-23 21:09:50,256] WARNING It is required that you pass in a value for the "algorithms" argument when calling decode().
[2023-01-23 21:09:50,258] INFO Reauthorize Result: Passed
[2023-01-23 21:09:50,603] INFO Authorization Status: None
[2023-01-23 21:09:50,604] INFO Authorization Method: Tokens


2020-10-23 07:48:30
/data/chime/intensity/processed/2020/10/23/astro_139459007
{'intensity-dm-pipeline': '2020-11-20T03:52:03.124000', 'intensity-fitburst': 1673134971.478}
Locked id 1673134971.478
1169
1169
{'event_id': 139459007, 'fitburst_snr': 12.37217054496588, 'fitburst_dm': 350.4014035760195, 'fitburst_dm_error': 0.02291134304379, 'num_sub_bursts': 1, 'pulse_width_ms': [0.002643597325383], 'pulse_width_ms_error': [0.00029773530662000004], 'scattering_time_ms': 0.005287194650766, 'scattering_time_ms_error': 0, 'fitburst_amplitude': [5.268810076356048e-05], 'fitburst_amplitude_error': [1.3552013396675692e-05], 'spectral_index': [1.901619786111957], 'spectral_index_error': [1.610992635653052], 'spectral_running': [-3.959472452646562], 'spectral_running_error': [2.38115063616277], 'Bandwidth': 0.0, 'Central Frequency': 0.0, 'UTC_400': ['2020-10-23 07:48:30.782062'], 'UTC_400_err': [0.001005728054863], 'type_of_data': 'intensity'} 

144520399


[2023-01-23 21:09:51,064] INFO Authorization Token : Expired
[2023-01-23 21:09:51,065] INFO Reauthorize Method: Tokens
[2023-01-23 21:09:51,918] WARNING It is required that you pass in a value for the "algorithms" argument when calling decode().
[2023-01-23 21:09:51,920] INFO Reauthorize Result: Passed
[2023-01-23 21:09:52,547] INFO Authorization Status: None
[2023-01-23 21:09:52,549] INFO Authorization Method: Tokens


...

2021-09-30 09:32:38
/data/chime/intensity/processed/2021/09/30/astro_188760957
{'intensity-fitburst': 1632995010.624}
Locked id 1632995010.624
169
169
{'event_id': 188760957, 'fitburst_snr': 11.332086272899804, 'fitburst_dm': 349.5296541942308, 'fitburst_dm_error': 0.055181739197075, 'num_sub_bursts': 1, 'pulse_width_ms': [0.0021743316133050004], 'pulse_width_ms_error': [0.00027823387784400004], 'scattering_time_ms': 0.004348663226610001, 'scattering_time_ms_error': 0, 'fitburst_amplitude': [2.7719201596842044e-05], 'fitburst_amplitude_error': [1.2202009820266612e-05], 'spectral_index': [27.05884903160525], 'spectral_index_error': [8.22071158992608], 'spectral_running': [-131.3564742663251], 'spectral_running_error': [37.41377188542152], 'Bandwidth': 0.0, 'Central Frequency': 0.0, 'UTC_400': ['2021-09-30 09:32:38.521022'], 'UTC_400_err': [0.001344123107755], 'type_of_data': 'intensity'} 

198142107


[2023-01-23 21:09:59,822] INFO Authorization Token : Expired
[2023-01-23 21:09:59,823] INFO Reauthorize Method: Tokens
[2023-01-23 21:10:00,339] WARNING It is required that you pass in a value for the "algorithms" argument when calling decode().
[2023-01-23 21:10:00,340] INFO Reauthorize Result: Passed
[2023-01-23 21:10:00,710] INFO Authorization Status: None
[2023-01-23 21:10:00,710] INFO Authorization Method: Tokens


2021-11-18 06:22:54
/data/chime/intensity/processed/2021/11/18/astro_198142107
{'intensity-fitburst': 1667330556.964, 'intensity-dm-pipeline': 1667330084.148}
Locked id 1667330556.964
1169
1169
{'event_id': 198142107, 'fitburst_snr': 14.129389627111546, 'fitburst_dm': 350.21475012766626, 'fitburst_dm_error': 0.04690739722823101, 'num_sub_bursts': 1, 'pulse_width_ms': [0.0036849673246810003], 'pulse_width_ms_error': [0.00037485593172900005], 'scattering_time_ms': 0.007369934649362001, 'scattering_time_ms_error': 0, 'fitburst_amplitude': [2.055788006008318e-05], 'fitburst_amplitude_error': [9.065563171975668e-06], 'spectral_index': [21.735708727071508], 'spectral_index_error': [4.900103704355601], 'spectral_running': [-60.85367673721211], 'spectral_running_error': [13.276061216801958], 'Bandwidth': 0.0, 'Central Frequency': 0.0, 'UTC_400': ['2021-11-18 06:22:54.953842'], 'UTC_400_err': [0.0024389469286710003], 'type_of_data': 'intensity'} 

201096712


[2023-01-23 21:10:01,022] INFO Authorization Token : Expired
[2023-01-23 21:10:01,023] INFO Reauthorize Method: Tokens
[2023-01-23 21:10:01,569] WARNING It is required that you pass in a value for the "algorithms" argument when calling decode().
[2023-01-23 21:10:01,570] INFO Reauthorize Result: Passed


2021-12-05 05:01:59
/data/chime/intensity/processed/2021/12/05/astro_201096712
{'intensity-dm-pipeline': 1674501805.648, 'intensity-fitburst': 1674507245.677}
Locked id 1674507245.677
1169
1169
{'event_id': 201096712, 'fitburst_snr': 7.6775055404276635, 'fitburst_dm': 349.24552503509335, 'fitburst_dm_error': 0.015856226037773, 'num_sub_bursts': 1, 'pulse_width_ms': [0.0009999999], 'pulse_width_ms_error': [0.000185558901666], 'scattering_time_ms': 0.0019999998, 'scattering_time_ms_error': 0, 'fitburst_amplitude': [2.329562615305814e-05], 'fitburst_amplitude_error': [8.954600840376394e-06], 'spectral_index': [-0.09156356017540901], 'spectral_index_error': [2.5704288483414492], 'spectral_running': [-1.146494757051617], 'spectral_running_error': [3.833635157811746], 'Bandwidth': 0.0, 'Central Frequency': 0.0, 'UTC_400': ['2021-12-05 05:01:59.850211'], 'UTC_400_err': [0.007311715784967], 'type_of_data': 'intensity'} 



In [7]:
##### Here timestamp is taken using different notebook get TOA ######
tstamps = [['2021-02-14 00:12:42.949611'], ['2021-03-01 23:21:50.418838'], ['2021-03-02 23:28:27.391257'], 
           ['2021-03-20 21:41:56.454417'], ['2021-06-26 15:52:16.182347'], ['2021-07-11 14:32:49.256215']]
tstamps = np.array(tstamps)
for i in range(len(R3_e_fon)):
    event_id = R3_e_fon[i]
    file  = '/data/user-data/efonseca/runs/R3/solutions/with_scattering/results_fitburst_{}.json'.format(event_id)
    data = pd.read_json(file)

    R3_fitburst_event = {'event_id': event_id,
                         'fitburst_snr': 0,
                         'fitburst_dm': 0,
                         'fitburst_dm_error':0.0,
                         'num_sub_bursts':0,
                         'pulse_width_ms': [0],
                         'pulse_width_ms_error': [0],
                         'scattering_time_ms': 0,
                         'scattering_time_ms_error': 0,
                         'fitburst_amplitude': [0],
                         'fitburst_amplitude_error': [0],
                         'spectral_index': [0],
                         'spectral_index_error': [0],
                         'spectral_running': [0],
                         'spectral_running_error':[0],
                         'Bandwidth':0.0,
                         'Central Frequency':0.0,
                         'UTC_400': 0.0,
                         'UTC_400_err':0.0,
                         'type_of_data': 'intensity'
                        }


    R3_fitburst_event['fitburst_snr'] = data["fit_statistics"]["snr"]
    R3_fitburst_event['fitburst_dm'] = data["model_parameters"]["dm"][0]
    R3_fitburst_event['num_sub_bursts'] = len(data["fit_statistics"]["bestfit_parameters"]["burst_width"])
    R3_fitburst_event['pulse_width_ms'] = data["fit_statistics"]["bestfit_parameters"]["burst_width"]
    R3_fitburst_event['pulse_width_ms_error'] = data["fit_statistics"]["bestfit_uncertainties"]["burst_width"]
    R3_fitburst_event['scattering_time_ms'] = data["fit_statistics"]["bestfit_parameters"]["scattering_timescale"][0]
    R3_fitburst_event['scattering_time_ms_error'] = data["fit_statistics"]["bestfit_uncertainties"]["scattering_timescale"][0]
    R3_fitburst_event['fitburst_amplitude'] = data["fit_statistics"]["bestfit_parameters"]["amplitude"]
    R3_fitburst_event['fitburst_amplitude_error'] = data["fit_statistics"]["bestfit_uncertainties"]["amplitude"]
    R3_fitburst_event['Central Frequency'] = data["fit_statistics"]["bestfit_parameters"]["freq_mean"]
    R3_fitburst_event['Bandwidth'] = data["fit_statistics"]["bestfit_parameters"]["freq_width"]
    R3_fitburst_event['UTC_400'] = tstamps[i]

    print(R3_fitburst_event, "\n")
    All_event_data.append(R3_fitburst_event)

{'event_id': 157779605, 'fitburst_snr': 18.418199218993657, 'fitburst_dm': 348.772, 'fitburst_dm_error': 0.0, 'num_sub_bursts': 1, 'pulse_width_ms': [0.001779030418484], 'pulse_width_ms_error': [8.123560044916047e-06], 'scattering_time_ms': 0.002662012109273, 'scattering_time_ms_error': 7.310451091642492e-06, 'fitburst_amplitude': [-3.869625612528428], 'fitburst_amplitude_error': [0.029384464439283], 'spectral_index': [0], 'spectral_index_error': [0], 'spectral_running': [0], 'spectral_running_error': [0], 'Bandwidth': [40.37254951976685], 'Central Frequency': [668.0375455002746], 'UTC_400': array(['2021-02-14 00:12:42.949611'], dtype='<U26'), 'UTC_400_err': 0.0, 'type_of_data': 'intensity'} 

{'event_id': 162981697, 'fitburst_snr': 3.928380589899192, 'fitburst_dm': 348.772, 'fitburst_dm_error': 0.0, 'num_sub_bursts': 1, 'pulse_width_ms': [0.001832425914638], 'pulse_width_ms_error': [0.00033755373790000004], 'scattering_time_ms': 0.002302564548625, 'scattering_time_ms_error': 0.0006174

In [8]:
import csv
field_names = ['event_id','fitburst_snr','fitburst_dm','fitburst_dm_error','num_sub_bursts','pulse_width_ms',
              'pulse_width_ms_error','scattering_time_ms','scattering_time_ms_error','fitburst_amplitude',
              'fitburst_amplitude_error','spectral_index','spectral_index_error','spectral_running',
              'spectral_running_error','Bandwidth','Central Frequency','UTC_400','UTC_400_err',
              'type_of_data']
with open('R3_intensity_pub.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(All_event_data)

In [28]:
from iautils import scripts

In [ ]:
timestamps = scripts.compute_timestamps.compute_timestamps([eventid], ftest_threshold=0.001, precision=5, 
                                                    rajd=catalog_event["ra"], decjd=catalog_event["dec"])
catalog_event['toa'] = timestamps[eventid]["fitburst_timestamp_UTC_MJD"]
catalog_event['toa_error'] = timestamps[eventid]["fitburst_mjd_err"]
catalog_event['toa_inf'] = timestamps[eventid]["corrected_timestamp_infinite_freq_MJD"]
catalog_event['toa_inf_error'] = timestamps[eventid]["corrected_timestamp_inf_MJD_err"]
if complete["localization"]:
    catalog_event['toas_inf'] = timestamps[eventid]["corrected_timestamp_infinite_freq_MJD_bary"]
complete["toas"] = True